In [21]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV

In [22]:
path="cleaned_imdb_sample.csv"
df = pd.read_csv(path)
df

,review_text,sentiment,cleaned_review
0,In Panic In The Streets Richard Widmark plays ...,positive,panic street richard widmark play navy doctor ...
1,If you ask me the first one was really better ...,negative,ask first one really better one look sarah rea...
2,I am a big fan a Faerie Tale Theatre and I've ...,positive,big fan faerie tale theatre ive seen one best ...
3,I just finished reading a book about Dillinger...,negative,finished reading book dillinger movie horribly...
4,Greg Davis and Bryan Daly take some crazed sta...,negative,greg davis bryan daly take crazed statement te...
...,...,...,...
9995,Many have stated that Orca  Killer Whale is a...,negative,many stated orca killer whale jaw ripoff reall...
9996,What a wasted cast.<br /><br />This is one of ...,negative,wasted castthis one disappointing film ive see...
9997,"First of all the movie, is an ingenious work o...",positive,first movie ingenious work artmovie plot fille...
9998,Wow. Saw this last night and I'm still reeling...,positive,wow saw last night still reeling good every ch...


In [23]:
df = df[['cleaned_review', 'sentiment']]
df.head(7)

,cleaned_review,sentiment
0,panic street richard widmark play navy doctor ...,positive
1,ask first one really better one look sarah rea...,negative
2,big fan faerie tale theatre ive seen one best ...,positive
3,finished reading book dillinger movie horribly...,negative
4,greg davis bryan daly take crazed statement te...,negative
5,really incredible film document eternal strugg...,positive
6,lived film time painful time quite joyous smal...,positive


In [34]:
from sklearn.model_selection import train_test_split

# Define features and target
features = df['cleaned_review']
target = df['sentiment']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    features, target, 
    test_size=0.2, 
    random_state=42, 
    stratify=target
)

In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize TF-IDF vectorizer with limited features
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_train_vectorized = tfidf_vectorizer.fit_transform(X_train)
X_test_vectorized = tfidf_vectorizer.transform(X_test)

In [36]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# Initialize classification models with optimized parameters
logistic_model = LogisticRegression(max_iter=1000, random_state=42, C=1.0)
decision_tree = DecisionTreeClassifier(random_state=42, max_depth=10)
random_forest = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=15)

# Train the machine learning models
logistic_model.fit(X_train_vectorized, y_train)
decision_tree.fit(X_train_vectorized, y_train)
random_forest.fit(X_train_vectorized, y_train)

,n_estimators,100
,criterion,'gini'
,max_depth,15
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [37]:
from sklearn.metrics import classification_report, accuracy_score
import pandas as pd

# Make sure these models have been trained first
models = {'Logistic Regression': logistic_model, 'Decision Tree': decision_tree, 'Random Forest': rf}
results = []

for name, model in models.items():
    y_pred = model.predict(X_test_tfidf)
    acc = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred, output_dict=True)
    avg = report['weighted avg']
    results.append({
        'Model': name,
        'Accuracy': f"{acc:.4f}",
        'Precision': f"{avg['precision']:.4f}",
        'Recall': f"{avg['recall']:.4f}",
        'F1': f"{avg['f1-score']:.4f}"
    })

comparison_df = pd.DataFrame(results)
comparison_df

,Model,Accuracy,Precision,Recall,F1
0,Logistic Regression,0.8380,0.8382,0.8380,0.8380
1,Decision Tree,0.7230,0.7364,0.7230,0.7186
2,Random Forest,0.8165,0.8170,0.8165,0.8165


In [38]:
import random

sample_idx = random.sample(range(len(X_test)), 5)
sample_reviews = X_test.iloc[sample_idx]
sample_labels = y_test.iloc[sample_idx]

output_lines = []  

print("=== Example Predictions ===\n")
output_lines.append("=== Example Predictions ===\n")

for i, (review, true_label) in enumerate(zip(sample_reviews, sample_labels), 1):
    header = f"Review {i}:"
    print(header)
    output_lines.append(header)
    
    review_text = review if len(review) <= 300 else review[:300] + "..."
    print("Review Text:", review_text)
    output_lines.append(f"Review Text: {review_text}")
    
    print("True Sentiment:", true_label)
    output_lines.append(f"True Sentiment: {true_label}")
    
    for name, model in models.items():
        pred = model.predict(tfidf_vectorizer.transform([review]))[0]
        print(f"{name} Prediction: {pred}")
        output_lines.append(f"{name} Prediction: {pred}")
    
    print("-" * 80)
    output_lines.append("-" * 80)

with open("example_predictions.txt", "w", encoding="utf-8") as f:
    for line in output_lines:
        f.write(line + "\n")

=== Example Predictions ===

Review 1:
Review Text: gypo nolan victor mclaglen poor anyone earth living ireland gypo fellow irishman part underground rebellion oppressive brit one particular rebel wanted murder english arrives back town secretly think trust friend gypo reward prof tempting gypo get friend killed sink pit despair drunkenness meanwhile...
True Sentiment: positive
Logistic Regression Prediction: positive
Decision Tree Prediction: negative
Random Forest Prediction: positive
--------------------------------------------------------------------------------
Review 2:
Review Text: finished reading book anita loo work photo tcm magazine macdonald angel costume looked great impressive wing thought watch movie never heard film preconceived notion whatsoever thought got cute start eddy playboy macdonald secretary doesnt know exists scene show costume party simple angel outfit unc...
True Sentiment: negative
Logistic Regression Prediction: positive
Decision Tree Prediction: positive

### svm (support victor machine)

In [40]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

# Initialize and train SVM
svm_model = SVC(kernel='linear', random_state=42)  # Linear kernel works well for text data
svm_model.fit(X_train_tfidf, y_train)

# Add SVM to your models dictionary
models['SVM'] = svm_model

# Evaluate SVM
svm_pred = svm_model.predict(X_test_tfidf)
svm_acc = accuracy_score(y_test, svm_pred)
svm_report = classification_report(y_test, svm_pred, output_dict=True)
svm_avg = svm_report['weighted avg']

# Update results with SVM
results.append({
    'Model': 'SVM',
    'Accuracy': f"{svm_acc:.4f}",
    'Precision': f"{svm_avg['precision']:.4f}",
    'Recall': f"{svm_avg['recall']:.4f}",
    'F1': f"{svm_avg['f1-score']:.4f}"
})

# Create updated comparison dataframe
comparison_df = pd.DataFrame(results)
comparison_df = comparison_df.sort_values('F1', ascending=False)
comparison_df

,Model,Accuracy,Precision,Recall,F1
0,Logistic Regression,0.8380,0.8382,0.8380,0.8380
3,SVM,0.8295,0.8295,0.8295,0.8295
2,Random Forest,0.8165,0.8170,0.8165,0.8165
1,Decision Tree,0.7230,0.7364,0.7230,0.7186


In [ ]:
#For hyperparameter tuning
from sklearn.model_selection import GridSearchCV

# SVM with hyperparameter tuning
svm_param_grid = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto']
}

svm_grid = GridSearchCV(SVC(random_state=42), svm_param_grid, cv=5, scoring='f1_weighted')
svm_grid.fit(X_train_tfidf, y_train)

print("Best SVM parameters:", svm_grid.best_params_)

# Use the best SVM model
best_svm = svm_grid.best_estimator_
models['SVM (Tuned)'] = best_svm

# Evaluate tuned SVM
svm_tuned_pred = best_svm.predict(X_test_tfidf)
svm_tuned_acc = accuracy_score(y_test, svm_tuned_pred)
svm_tuned_report = classification_report(y_test, svm_tuned_pred, output_dict=True)
svm_tuned_avg = svm_tuned_report['weighted avg']

results.append({
    'Model': 'SVM (Tuned)',
    'Accuracy': f"{svm_tuned_acc:.4f}",
    'Precision': f"{svm_tuned_avg['precision']:.4f}",
    'Recall': f"{svm_tuned_avg['recall']:.4f}",
    'F1': f"{svm_tuned_avg['f1-score']:.4f}"
})

# Final comparison
final_comparison_df = pd.DataFrame(results)
final_comparison_df = final_comparison_df.sort_values('F1', ascending=False)
final_comparison_df